In [1]:
import pyprind
import pandas as pd
from string import punctuation
import re
import numpy as np

In [2]:
df = pd.read_csv('movie_data.csv',encoding='utf-8')

In [3]:
from collections import Counter
c = Counter()
c.update('abc')

In [4]:
c

Counter({'a': 1, 'b': 1, 'c': 1})

In [5]:
c['a']

1

In [6]:
c.update(['a','b'])

In [7]:
c

Counter({'a': 2, 'b': 2, 'c': 1})

In [8]:
c.update({'c':3})

In [9]:
c

Counter({'a': 2, 'b': 2, 'c': 4})

In [12]:
c.most_common(1)#가장 최빈 한 개 출력

[('c', 4)]

In [13]:
c.most_common()

[('c', 4), ('a', 2), ('b', 2)]

In [14]:
c.most_common(2)#최빈값 2개 출력

[('c', 4), ('a', 2)]

In [17]:
#데이터 전처리
#단어를 나누고 등장 횟수를 카운트함
from collections import Counter#원소를 카운트하여 딕셔너리에 저장
counts = Counter()
pbar = pyprind.ProgBar(len(df['review']),
                      title = '단어의 등장 횟수를 카운트합니다')

for i, review in enumerate(df['review']):
    text = ''.join([c if c not in punctuation else ' '+c+' '
                   for c in review] ).lower()
    df.loc[i, 'review'] = text
    pbar.update()
    counts.update(text.split())

단어의 등장 횟수를 카운트합니다
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:06:24


In [19]:
#고유한 각 단어를 정수로 매핑하는
#딕셔너리 생성
word_counts = sorted(counts, key=counts.get, reverse = True)
print(word_counts[:5])
word_to_int = {word:ii for ii,word in
              enumerate(word_counts,1)}#전체 리뷰 텍스트를 정수리스트로 변환

['the', '.', ',', 'and', 'a']


In [21]:
mapped_reviews = []
pbar = pyprind.ProgBar(len(df['review']),
                      title = '리뷰를 정수로 매핑합니다')
for review in df['review']:
    mapped_reviews.append([word_to_int[word]
                          for word in review.split()])
    pbar.update()

리뷰를 정수로 매핑합니다
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


In [22]:
#동일 길이의 시퀀스를 만듭니다
#시퀀스 길이가 200보다 작으면 왼쪽에 0이 패딩됩니다
#시퀀스 길이가 200보다 크면 마지막 200개 원소만 사용합니다(첫 원소들 잘림)
#데이터셋 전처리

sequence_length = 200#RNN 시퀀스 길이
sequences = np.zeros((len(mapped_reviews), sequence_length),
                    dtype=int)#0으로 채우기

for i, row in enumerate(mapped_reviews):
    review_arr = np.array(row)
    sequences[i, -len(row):] = review_arr[-sequence_length:]#동일한 길이의 시퀀스 만들기

In [23]:
#훈련 세트와 테스트 세트로 나눔
X_train = sequences[:37500,:]#75% 훈련세트
y_train = df.loc[:37499, 'sentiment'].values
X_test = sequences[37500:,:]#25% 테스트 세트
y_test = df.loc[37500:, 'sentiment'].values

In [24]:
#크기 확인
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(37500, 200) (37500,) (12500, 200) (12500,)


In [25]:
#딕셔너리 크기 확인
n_words = len(word_to_int)+1#딕셔너리 크기 확인
print(n_words)

102967


In [26]:
#임베딩: 데이터셋에 있는 필터를 표현하는데 중요한 특성을 자동으로 학습 가능
from tensorflow.keras import modelsls, layers
model = models.Sequential()

In [27]:
#임베딩층 추가
model.add(layers.Embedding(n_words, 200,#어휘 사전의 크기, 출력 차원
                          embeddings_regularizer = 'l2'))#L2 규제

In [29]:
#모델 구조 출력
model.summary()#임베딩의 출력은 3차원 텐서(배치 차원, 타임 스템, 임베딩 벡터의 차원)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 200)         20593400  
Total params: 20,593,400
Trainable params: 20,593,400
Non-trainable params: 0
_________________________________________________________________


In [30]:
#LSTM층 사용
model.add(layers.LSTM(16))#16개의 순환 유닛